In [57]:
import pandas as pd
import numpy as np


## Data Reduction Process:

In [58]:
df = pd.read_csv("data/min_colon.csv")
# ndata = df
df.head()
# ndata.transpose()

,x67699,r70790,l11706,t90549,d17390,m33210,h18490,h40891,r77780,t49647,class
0,99.11072,67.56125,259.91250,138.89874,88.23250,39.66786,67.82875,75.67750,83.52250,28.70125,Tumor
1,126.78929,92.23875,96.27625,150.59000,82.23750,85.03333,152.19500,186.56750,44.47250,16.77375,Normal
2,151.87738,82.71500,31.10250,193.92000,76.97250,224.62024,31.22500,42.65625,16.09250,15.15625,Tumor
3,152.59525,41.68375,5.92500,183.00626,74.52875,67.71072,48.33875,42.52000,49.98250,16.08500,Normal
4,126.46429,76.60375,161.35000,61.70125,54.56375,223.35953,73.09875,57.59875,7.48875,31.81250,Tumor


In [59]:
# Eliminating the 'Normally' expressed genes.
data = df.drop(df[(df['class'] == 'Normal')].index, inplace=True)
data = df
df.to_csv('data/removed_normal_dataset.csv', index=False)
data.head()
# print(type(data))


,x67699,r70790,l11706,t90549,d17390,m33210,h18490,h40891,r77780,t49647,class
0,99.11072,67.56125,259.91250,138.89874,88.23250,39.66786,67.82875,75.67750,83.52250,28.70125,Tumor
2,151.87738,82.71500,31.10250,193.92000,76.97250,224.62024,31.22500,42.65625,16.09250,15.15625,Tumor
4,126.46429,76.60375,161.35000,61.70125,54.56375,223.35953,73.09875,57.59875,7.48875,31.81250,Tumor
6,147.80476,51.03250,80.19000,76.48500,98.53750,54.62143,30.54000,52.96125,37.23625,24.44500,Tumor
8,111.13810,42.14625,277.85376,176.10374,149.15625,209.91190,99.91500,122.44125,40.39125,26.84750,Tumor


In [60]:
# transposing the dataframe and droping the 'class'
# data = df.drop(['class'], axis=1)
data = df.transpose().drop(['class'], axis = 0)
# data = data.transpose()
# change the name of the column as 'Assay01, Assay02, Assay03....'
col = data.columns
colNames = list()
for i in range(len(col)):
    colNames.append('Assay{}'.format(i))
# changing the column names
data.columns = colNames
data.to_csv('data/transposed_data.csv')
data.head()


,Assay0,Assay1,Assay2,Assay3,Assay4,Assay5,Assay6,Assay7,Assay8,Assay9,...,Assay30,Assay31,Assay32,Assay33,Assay34,Assay35,Assay36,Assay37,Assay38,Assay39
x67699,99.11072,151.87738,126.46429,147.80476,111.1381,306.47263,185.71786,254.91667,5.95,80.97024,...,246.32619,192.76071,146.90356,209.34525,109.1869,152.85,270.9357,70.667854,129.40475,190.7119
r70790,67.56125,82.715,76.60375,51.0325,42.14625,271.79376,112.21875,64.88,26.715,29.935,...,185.83125,107.795,77.60625,105.1975,32.20375,88.99625,28.98,6.4475,35.87875,85.25875
l11706,259.9125,31.1025,161.35,80.19,277.85376,554.1937,189.05376,435.98126,244.27,155.7675,...,720.59375,624.7275,162.0225,902.5725,601.8475,203.76,530.16876,156.445,342.65,206.53625
t90549,138.89874,193.92,61.70125,76.485,176.10374,65.1975,80.5,144.28,30.775,50.895,...,154.1475,155.8225,114.8275,128.015,117.00375,114.48875,264.92,93.95,84.75125,80.44875
d17390,88.2325,76.9725,54.56375,98.5375,149.15625,277.2075,93.9075,71.16125,23.96875,43.6575,...,278.575,173.105,98.47125,203.285,62.32,113.5225,79.34125,86.90875,60.27375,104.805


### Calculating the Relative Gene Expression Levels:
- Arbitrary pick a reference assay and calculate the relative expression levels based on the reference assay.
- Assuming the absolute gene expression levels of the reference assay is: ***`Er1, Er2, Er3,...,Erm`***
- Calculate the relative gene expression levels for another assay ***`A`*** as: ***`Ra1, Ra2, Ra3, Ra4,....,Ran = Er1/Ea1, Er2/Ea2,...Erm/Ean`***
- Calculate for all the rests.

In [61]:
new_data = pd.read_csv("data/transposed_data.csv")
new_data.rename(columns={"Unnamed: 0":"GeneNames"}, inplace=True)
new_data.head()

,GeneNames,Assay0,Assay1,Assay2,Assay3,Assay4,Assay5,Assay6,Assay7,Assay8,...,Assay30,Assay31,Assay32,Assay33,Assay34,Assay35,Assay36,Assay37,Assay38,Assay39
0,x67699,99.11072,151.87738,126.46429,147.80476,111.13810,306.47263,185.71786,254.91667,5.95000,...,246.32619,192.76071,146.90356,209.34525,109.18690,152.85000,270.93570,70.667854,129.40475,190.71190
1,r70790,67.56125,82.71500,76.60375,51.03250,42.14625,271.79376,112.21875,64.88000,26.71500,...,185.83125,107.79500,77.60625,105.19750,32.20375,88.99625,28.98000,6.447500,35.87875,85.25875
2,l11706,259.91250,31.10250,161.35000,80.19000,277.85376,554.19370,189.05376,435.98126,244.27000,...,720.59375,624.72750,162.02250,902.57250,601.84750,203.76000,530.16876,156.445000,342.65000,206.53625
3,t90549,138.89874,193.92000,61.70125,76.48500,176.10374,65.19750,80.50000,144.28000,30.77500,...,154.14750,155.82250,114.82750,128.01500,117.00375,114.48875,264.92000,93.950000,84.75125,80.44875
4,d17390,88.23250,76.97250,54.56375,98.53750,149.15625,277.20750,93.90750,71.16125,23.96875,...,278.57500,173.10500,98.47125,203.28500,62.32000,113.52250,79.34125,86.908750,60.27375,104.80500


In [62]:
# selecting 'Assay62' as the reference assay
refAssay = new_data['Assay39']
# droping the reference assay to reduce the dataset size
new_data = new_data.drop(['Assay39'], axis=1)

In [63]:
# Calculating the Relative Gene Expression Levels w.r.t the Reference Gene
for i in range(len(col)-1):
    new_data['Assay{}'.format(i)] = new_data['Assay{}'.format(i)].div(refAssay)

new_data.to_csv("data/relExp_data.csv")
relExpr = new_data
relExpr.head()

,GeneNames,Assay0,Assay1,Assay2,Assay3,Assay4,Assay5,Assay6,Assay7,Assay8,...,Assay29,Assay30,Assay31,Assay32,Assay33,Assay34,Assay35,Assay36,Assay37,Assay38
0,x67699,0.519688,0.796371,0.663117,0.775016,0.582754,1.606993,0.973814,1.336658,0.031199,...,3.803319,1.291614,1.010743,0.770290,1.097704,0.572523,0.801471,1.420654,0.370548,0.678535
1,r70790,0.792426,0.970164,0.898485,0.598560,0.494333,3.187869,1.316214,0.760978,0.313340,...,3.306435,2.179615,1.264328,0.910244,1.233862,0.377718,1.043837,0.339906,0.075623,0.420822
2,l11706,1.258435,0.150591,0.781219,0.388261,1.345303,2.683276,0.915354,2.110919,1.182698,...,2.530222,3.488946,3.024784,0.784475,4.370044,2.914004,0.986558,2.566953,0.757470,1.659031
3,t90549,1.726549,2.410479,0.766963,0.950730,2.189018,0.810423,1.000637,1.793440,0.382542,...,4.105890,1.916096,1.936916,1.427337,1.591262,1.454389,1.423127,3.293028,1.167824,1.053481
4,d17390,0.841873,0.734435,0.520622,0.940198,1.423179,2.644984,0.896021,0.678987,0.228699,...,2.799974,2.658032,1.651686,0.939566,1.939650,0.594628,1.083178,0.757037,0.829242,0.575104


## Calculating the Log-Scale Gene Expression Levels:
- Log-Scale Gene Expressio Levels are calculated as: ***`La1, La2, La3,...Lan = logRa1, logRa2,...logRan`***

In [64]:
# changing the relative gene expression levels into log-scale gene expression levels.
for i in range(len(col)-1):
    relExpr['Assay{}'.format(i)] = np.log2(relExpr['Assay{}'.format(i)])
relExpr.head(10)


,GeneNames,Assay0,Assay1,Assay2,Assay3,Assay4,Assay5,Assay6,Assay7,Assay8,...,Assay29,Assay30,Assay31,Assay32,Assay33,Assay34,Assay35,Assay36,Assay37,Assay38
0,x67699,-0.944282,-0.328488,-0.592665,-0.367702,-0.779041,0.684363,-0.038282,0.418631,-5.002361,...,1.927259,0.369175,0.015416,-0.376526,0.134489,-0.804595,-0.319278,0.506556,-1.432269,-0.559504
1,r70790,-0.335652,-0.043699,-0.154433,-0.740432,-1.016444,1.672593,0.396394,-0.394074,-1.674198,...,1.725276,1.124073,0.338370,-0.135675,0.303181,-1.404619,0.061897,-1.556790,-3.725036,-1.248718
2,l11706,0.331631,-2.731293,-0.356201,-1.364901,0.427931,1.423995,-0.127598,1.077871,0.242082,...,1.339264,1.802791,1.596832,-0.350201,2.127648,1.543003,-0.019524,1.360057,-0.400739,0.730341
3,t90549,0.787892,1.269320,-0.382770,-0.072893,1.130284,-0.303253,0.000919,0.842729,-1.386311,...,2.037695,0.938170,0.953762,0.513326,0.670171,0.540413,0.509064,1.719415,0.223823,0.075165
4,d17390,-0.248325,-0.445293,-0.941693,-0.088963,0.509117,1.403259,-0.158395,-0.558544,-2.128481,...,1.485413,1.410358,0.723940,-0.089933,0.955796,-0.749940,0.115271,-0.401565,-0.270134,-0.798106
5,m33210,-2.189482,0.311963,0.303843,-1.727986,0.214259,1.291084,-0.161218,0.233449,-0.316372,...,-0.271204,0.743793,0.717301,-0.698373,-0.038155,-0.113015,0.010262,1.822679,-0.922816,-0.330685
6,h18490,-0.023314,-1.142509,0.084636,-1.174510,0.535491,2.454488,-1.247107,-0.421724,-0.585734,...,1.803429,2.071109,1.730836,0.251029,-0.045389,-2.377314,-1.254952,-3.294384,-1.144243,-2.937815
7,h40891,-0.643309,-1.470416,-1.037136,-1.158236,0.050844,1.503594,-1.377228,-0.436345,-1.011996,...,1.064988,1.496842,0.253132,0.309409,0.358965,-1.182098,-0.565023,-0.322325,-1.048357,-0.145270
8,r77780,1.353426,-1.022351,-2.125942,0.187969,0.305304,1.835499,-0.271911,0.551249,-0.921620,...,2.592995,1.569933,1.023394,1.149341,1.796358,0.545779,-0.156423,0.798041,-1.227440,-0.494459
9,t49647,0.302952,-0.618248,0.451433,0.071378,0.206627,1.334308,0.576463,1.942981,1.059600,...,2.038793,2.272826,1.661937,-0.703155,1.494325,0.386859,0.832732,0.430304,-1.141260,0.467387


In [65]:
# Exporting the Log-Scaled Values as a csv file
relExpr.to_csv('data/log_scaled_data.csv', index=False)

In [66]:
# # Correaltion analysis of the log-scaled dataset
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

# corrMat = relExpr.corr()
# f= plt.figure(figsize=(20,20))
# sns.heatmap(corrMat, cmap="coolwarm", linewidth=0.1, annot=True)
# # Correlation analysis is not required here